In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from PIL import Image
from src.train_haze import Dataset, double_conv, UNet
import cv2
from matplotlib.pyplot import imshow

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from src.preprocessing.trans_func import *

In [5]:
model = UNet()
model = torch.nn.DataParallel(model).cuda()
model.load_state_dict(torch.load("../src/model_haze_china.pth"),strict=False) # on GPU
criterion = nn.MSELoss().cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)

In [10]:
transform = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(),transforms.Normalize(mean=[0.5231, 0.5180, 0.5115],std=[0.2014, 0.2018, 0.2100]),])

imageName = 'test_shanghai_1.jpg'

cv_in = cv2.imread(imageName,1) # color
pil_in = Image.fromarray(cv_in)
X = transform(pil_in)

X = X[None, :, :, :]

x = X.cuda(non_blocking=True)
x_var = torch.autograd.Variable(x)

pred_haze = model(x_var)

transform2 = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(),])

img = cv2.imread(imageName)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
light_intensity = find_intensity_of_atmospheric_light(img, gray)
w = 0.9
t0 = 0.01
outimg = dehaze(img, light_intensity, 20, t0, w)

pil_out = Image.fromarray(outimg)
Y = transform2(pil_out)
Y = Y[None, :, : :]
Y = Y.cuda(non_blocking=True)

In [11]:
Y

tensor([[[[0.8078, 0.7843, 0.7725,  ..., 0.7412, 0.7451, 0.7686],
          [0.8157, 0.7922, 0.7804,  ..., 0.7451, 0.7451, 0.7686],
          [0.8235, 0.8000, 0.7922,  ..., 0.7451, 0.7451, 0.7686],
          ...,
          [0.5412, 0.5294, 0.5294,  ..., 0.4863, 0.4863, 0.4784],
          [0.5608, 0.5333, 0.5216,  ..., 0.4745, 0.4471, 0.5098],
          [0.5451, 0.5255, 0.5137,  ..., 0.7137, 0.5569, 0.5059]]]],
       device='cuda:0')

In [12]:
pred_haze

tensor([[[[0.1534, 0.1753, 0.1818,  ..., 0.2336, 0.2197, 0.1838],
          [0.1485, 0.1664, 0.1779,  ..., 0.2371, 0.2246, 0.1992],
          [0.1616, 0.1782, 0.1743,  ..., 0.2384, 0.2302, 0.2059],
          ...,
          [0.4658, 0.4780, 0.4568,  ..., 0.4925, 0.4966, 0.4650],
          [0.4339, 0.4710, 0.4697,  ..., 0.4984, 0.5247, 0.4373],
          [0.3660, 0.4386, 0.4587,  ..., 0.2778, 0.3684, 0.3470]]]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [13]:
criterion(pred_haze, Y)

tensor(0.1729, device='cuda:0', grad_fn=<MseLossBackward>)